<a href="https://colab.research.google.com/github/evildead23151/F1-Prediction-Models/blob/main/Practice_Prediction_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# F1 Race Winners & Timings Prediction
## Using Gradient Booster

In [2]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [4]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [7]:
import os

if not os.path.exists("f1_cache"):
    os.makedirs("f1_cache")

# Enable FastF1 caching

In [8]:
fastf1.Cache.enable_cache("f1_cache")

# Load FastF1 2024 Australian GP race session

In [10]:
session_2024 = fastf1.get_session(2024, 3, "R")
session_2024.load()

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.0]
INFO:fastf1.fastf1.core:Loading data for Australian Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
DEBUG:fastf1.ergast:Failed to parse timestamp '-1:57:37.891' in Ergastresponse.
req            INFO 	No cached data found for session_status_

# Extract lap times

In [11]:
laps_2024 = session_2024.laps[["Driver", "LapTime"]].copy()
laps_2024.dropna(subset = ["LapTime"], inplace = True)
laps_2024["LapTime (s)"] = laps_2024["LapTime"].dt.total_seconds()

# 2025 Qualifying Data

In [12]:
qualifying_2025 = pd.DataFrame({
    "Driver": ["Lando Norris", "Oscar Piastri", "Max Verstappen", "George Russell", "Yuki Tsunoda",
               "Alexander Albon", "Charles Leclerc", "Lewis Hamilton", "Pierre Gasly", "Carlos Sainz", "Fernando Alonso", "Lance Stroll"],
    "QualifyingTime (s)": [75.096, 75.180, 75.481, 75.546, 75.670,
                           75.737, 75.755, 75.973, 75.980, 76.062, 76.4, 76.5]
})

# Map full names to FastF1 3-letter codes

In [14]:
driver_mapping = {
    "Lando Norris": "NOR", "Oscar Piastri": "PIA", "Max Verstappen": "VER", "George Russell": "RUS",
    "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Charles Leclerc": "LEC", "Lewis Hamilton": "HAM",
    "Pierre Gasly": "GAS", "Carlos Sainz": "SAI", "Lance Stroll": "STR", "Fernando Alonso": "ALO"
}
qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

# Merge 2025 Qualifying Data with 2024 Race Data

In [15]:
merged_data = qualifying_2025.merge(laps_2024, left_on="DriverCode", right_on = "Driver")

# Use only "QualifyingTime (s)" as a feature

In [16]:
X = merged_data[["QualifyingTime (s)"]]
y = merged_data["LapTime (s)"]
if X.shape[0]==0:
  raise ValueError("Dataset is empty after preprocessing. Check data sources!")

# Train Gradient Boosting Model

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=39)
model = GradientBoostingRegressor(n_estimators=100, learning_rate = 0.1, random_state = 39)
model.fit(X_train, y_train)

GradientBoostingRegressor(random_state=39)

# Predict using 2025 qualifying times

In [19]:
predicted_lap_times = model.predict(qualifying_2025[["QualifyingTime (s)"]])
qualifying_2025["PredictedRaceTime (s)"] = predicted_lap_times

# Rank drivers by predicted race time

In [20]:
qualifying_2025 = qualifying_2025.sort_values(by="PredictedRaceTime (s)")

# Print final predictions

In [22]:
print("\n Predicted 2025 Chinese GP Winner\n")
print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])


 Predicted 2025 Chinese GP Winner

             Driver  PredictedRaceTime (s)
0      Lando Norris              82.712267
6   Charles Leclerc              83.079012
9      Carlos Sainz              83.621623
10  Fernando Alonso              83.871856
3    George Russell              83.886910
1     Oscar Piastri              84.330049
4      Yuki Tsunoda              84.417801
5   Alexander Albon              84.644262
2    Max Verstappen              85.187728
11     Lance Stroll              85.287535
8      Pierre Gasly              85.541400
7    Lewis Hamilton              86.098211


# Evaluate Model

In [23]:
y_pred = model.predict(X_test)
print(f"\n Model Error(MAE) : {mean_absolute_error(y_test, y_pred):.2f} seconds")


 Model Error(MAE) : 3.47 seconds
